# 🤖 Chatbot educativo — FAQ Educativo Sintético
Motores de recuperación:
1. TF-IDF + cosine (baseline, interpretable)
2. Sentence-BERT (semántico, opcional; con fallback a TF-IDF)
Datos: faq_educativo_sintetico.csv

## 1) Cargar datos

El sistema inicia cargando el archivo CSV que contiene las preguntas frecuentes educativas. Se verifica que el archivo exista y se lee utilizando pandas. Los datos contienen preguntas, respuestas y categorías temáticas que permiten organizar el conocimiento por materias.

In [1]:
import pandas as pd
from pathlib import Path
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

CSV_PATH = Path("faq_educativo_sintetico.csv")
assert CSV_PATH.exists(), "No se encontró faq_educativo_sintetico.csv."

faq = pd.read_csv(CSV_PATH)
print(f"✓ Datos cargados: {len(faq)} preguntas en {faq['refs'].nunique()} categorías")
print(f"  Categorías: {', '.join(faq['refs'].unique())}\n")

✓ Datos cargados: 50 preguntas en 9 categorías
  Categorías: Biología, Historia, Física, Medio ambiente, Matemáticas, Astronomía, Química, Geografía, Historia de la ciencia



## 2) TF‑IDF + cosine (baseline)

Este es el motor base del sistema. Utiliza una técnica de vectorización de texto que analiza la frecuencia de palabras y bigramas en las preguntas. Calcula qué tan similar es la consulta del usuario con cada pregunta almacenada mediante el coseno del ángulo entre vectores, devolviendo las más cercanas.

In [6]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2), min_df=1)
X = vectorizer.fit_transform(faq["question"].fillna(""))

def retrieve_tfidf(query, topk=5):
    """Recupera las top-k preguntas más similares usando TF-IDF"""
    qv = vectorizer.transform([query])
    sims = cosine_similarity(qv, X)[0]
    idx = np.argsort(sims)[::-1][:topk]
    return [(int(i), float(sims[i])) for i in idx]

## 3) Sentence‑BERT (opcional) con *fallback*

Es una mejora semántica que utiliza redes neuronales para entender el significado de las oraciones. A diferencia del TF-IDF que solo ve palabras, este motor comprende contexto y sinónimos. Si no está disponible por problemas de instalación, el sistema automáticamente usa el motor TF-IDF como respaldo.

In [7]:
model = None
sbert_ready = False

try:
    from sentence_transformers import SentenceTransformer
    print("⏳ Cargando modelo Sentence-BERT...")
    model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
    sbert_ready = True
    print("✓ SBERT cargado exitosamente\n")
except Exception as e:
    print(f"⚠ SBERT no disponible; se usará TF-IDF exclusivamente.")
    print(f"  Detalle: {e}\n")

if sbert_ready:
    emb_q = model.encode(faq["question"].tolist(), normalize_embeddings=True)
    
    def retrieve_sbert(query, topk=5):
        """Recupera usando embeddings semánticos de SBERT"""
        q = model.encode([query], normalize_embeddings=True)[0]
        sims = emb_q @ q
        idx = np.argsort(sims)[::-1][:topk]
        return [(int(i), float(sims[i])) for i in idx]
else:
    def retrieve_sbert(query, topk=5):
        """Fallback a TF-IDF si SBERT no está disponible"""
        return retrieve_tfidf(query, topk=topk)

2025-10-29 20:14:58.105774: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-29 20:14:59.219548: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-29 20:15:02.959653: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


⚠ SBERT no disponible; se usará TF-IDF exclusivamente.
  Detalle: Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.



## 4) Respuesta con umbrales

El sistema evalúa la confianza de cada respuesta comparando la similitud obtenida con dos umbrales predefinidos. Si la similitud es alta, responde con certeza. Si es media, sugiere la respuesta pero ofrece alternativas. Si es baja, muestra las opciones más cercanas para que el usuario reformule su pregunta.

In [8]:
THRESHOLD_CERT = 0.55   # Alta confianza
THRESHOLD_MAYBE = 0.40  # Confianza media

def responder(query, topk=5, metodo="auto"):
    """
    Responde a una consulta del usuario
    
    Args:
        query: Pregunta del usuario
        topk: Número de candidatos a recuperar
        metodo: "tfidf", "sbert" o "auto" (usa SBERT si está disponible)
    
    Returns:
        dict con status, match y candidatos
    """
    metodo = metodo.lower()
    
    # Seleccionar método de recuperación
    if metodo == "tfidf":
        rank = retrieve_tfidf(query, topk=topk)
    elif metodo == "sbert":
        rank = retrieve_sbert(query, topk=topk)
    else:  # auto
        rank = retrieve_sbert(query, topk=topk)
    
    if not rank:
        return {
            "status": "empty",
            "message": "Sin resultados",
            "candidatos": []
        }
    
    best_idx, best_sim = rank[0]
    
    # Formatear candidatos
    cand = [{
        "score": float(sim),
        "question": faq.iloc[i]["question"],
        "answer": faq.iloc[i]["answer"],
        "refs": faq.iloc[i]["refs"]
    } for i, sim in rank]
    
    # Evaluar confianza
    if best_sim >= THRESHOLD_CERT:
        return {
            "status": "ok",
            "match": cand[0],
            "candidatos": cand
        }
    elif best_sim >= THRESHOLD_MAYBE:
        return {
            "status": "maybe",
            "match": cand[0],
            "candidatos": cand
        }
    else:
        return {
            "status": "uncertain",
            "message": "No estoy seguro. ¿Te refieres a alguna de estas preguntas?",
            "candidatos": cand
        }

## 5) Chat CLI (opcional)

Proporciona una interacción conversacional en la terminal donde el usuario puede hacer preguntas en lenguaje natural. El sistema responde mostrando la respuesta, el nivel de confianza, la categoría educativa y sugerencias de preguntas relacionadas cuando es necesario. Incluye comandos para salir de forma amigable.

In [10]:
def chat():
    """Interfaz de chat interactiva en consola"""
    print("=" * 60)
    print("🤖 Chatbot Educativo — FAQ Sintético")
    print("=" * 60)
    print("Escribe 'salir', 'exit' o 'quit' para terminar.\n")
    
    while True:
        q = input("Tú: ").strip()
        
        if q.lower() in {"salir", "exit", "quit"}:
            print("\nBot: ¡Hasta luego! 👋\n")
            break
        
        if not q:
            continue
        
        r = responder(q)
        
        if r["status"] == "ok":
            m = r["match"]
            print(f"\n🟢 Bot (confianza alta: {m['score']:.2f}):")
            print(f"   {m['answer']}")
            print(f"   📚 Categoría: {m['refs']}\n")
            
        elif r["status"] == "maybe":
            m = r["match"]
            print(f"\n🟡 Bot (posible respuesta: {m['score']:.2f}):")
            print(f"   {m['answer']}")
            print(f"   📚 Categoría: {m['refs']}")
            print("\n   También podrías haber preguntado:")
            for s in r["candidatos"][1:3]:
                print(f"   • {s['question']}")
            print()
            
        else:
            print(f"\n🔴 Bot: {r['message']}")
            for s in r["candidatos"][:3]:
                print(f"   • ({s['score']:.2f}) {s['question']}")
            print()

# ============================================
# FORMA DE USAR (vease en los resultados cuando se ejecuta)
# ============================================

In [11]:
if __name__ == "__main__":
    # Pruebas rápidas
    print("📝 Pruebas del sistema:\n")
    
    ejemplos = [
        "¿Qué es la fotosíntesis?",
        "Cuéntame sobre Albert Einstein",
        "¿Cómo funciona el reciclaje?",
        "¿Qué es un número primo?"
    ]
    
    for pregunta in ejemplos:
        print(f"Pregunta: {pregunta}")
        resultado = responder(pregunta, topk=3)
        if resultado["status"] in ["ok", "maybe"]:
            m = resultado["match"]
            print(f"Respuesta ({m['score']:.2f}): {m['answer'][:80]}...")
        else:
            print(f"Estado: {resultado['status']}")
        print()
    
    print("\n" + "=" * 60)
    print("Para iniciar el chat interactivo, ejecuta: chat()")
    print("=" * 60)

📝 Pruebas del sistema:

Pregunta: ¿Qué es la fotosíntesis?
Respuesta (1.00): Es el proceso por el cual las plantas convierten la energía solar en energía quí...

Pregunta: Cuéntame sobre Albert Einstein
Respuesta (0.73): Fue un físico alemán que desarrolló la teoría de la relatividad....

Pregunta: ¿Cómo funciona el reciclaje?
Respuesta (0.88): Es el proceso de recolectar y transformar materiales usados en nuevos productos ...

Pregunta: ¿Qué es un número primo?
Respuesta (1.00): Es un número mayor que 1 que solo tiene dos divisores: él mismo y el 1....


Para iniciar el chat interactivo, ejecuta: chat()


In [12]:
chat()

🤖 Chatbot Educativo — FAQ Sintético
Escribe 'salir', 'exit' o 'quit' para terminar.



Tú:  que es la fotosintesis



🟡 Bot (posible respuesta: 0.42):
   Es la variedad de organismos vivos que existen en la Tierra.
   📚 Categoría: Medio ambiente

   También podrías haber preguntado:
   • ¿Qué es la velocidad?
   • ¿Qué es la historia?



Tú:  velocidad que es



🟢 Bot (confianza alta: 0.64):
   Es la magnitud física que indica la distancia recorrida por un cuerpo en un tiempo determinado.
   📚 Categoría: Física



...


🔴 Bot: No estoy seguro. ¿Te refieres a alguna de estas preguntas?
   • (0.00) ¿Qué es el calentamiento global?
   • (0.00) ¿Qué es una media aritmética?
   • (0.00) ¿Qué es la velocidad?



Tú:  tema de fisica



🔴 Bot: No estoy seguro. ¿Te refieres a alguna de estas preguntas?
   • (0.32) ¿Qué es el reciclaje de papel?
   • (0.31) ¿Qué es la ley de Ohm?
   • (0.31) ¿Qué es el teorema de Pitágoras?



Tú:  exit



Bot: ¡Hasta luego! 👋



In [7]:
pip install "sentence-transformers>=3.0,<4"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 3.6 MB/s eta 0:00:0000:0100:07
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.7 MB/s eta 0:00:0000:0100:06
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 6.1 MB/s eta 0:00:0000:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 4.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 4.2 MB/s eta 0:00:0000:0100:05
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 3.7 MB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 4.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 3.4 MB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/